In [36]:
from time import time
import mysql.connector as dbCon
import pandas as pd
import numpy as np
import os
import glob
import pandas as pd
import datetime
import math
import json
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import sys

conn = dbCon.connect(host="localhost", user="root", password="cqX*5gk6^hvNFPvE", database="dsp")

argc = len(sys.argv)
#if argc < 6:
    #exit()

reportType = 'moneySales'#sys.argv[1]
dInterval = 'weekly'#sys.argv[2]
reportFormat = 'table' #sys.argv[3]
dFrom = '2019-01-01'#sys.argv[4]
dTo = '2019-02-01'#sys.argv[5]

if reportType == 'moneySales':
    sql = "SELECT * FROM money_sales WHERE DayT BETWEEN %s AND %s"
    salesCursor = conn.cursor()
    param = (dFrom, dTo)
    salesCursor.execute(sql, param)
    salesResult = salesCursor.fetchall()
    salesCursor.close()
    moneyDF = pd.DataFrame(salesResult, columns=['Date', 'Product', 'Quantity', 'Value'])
    moneyDF['Date'] = pd.to_datetime(moneyDF['Date'])

    if dInterval == 'daily':
        dailyDF = moneyDF.groupby('Date')['Value'].sum().reset_index()
        if reportFormat == 'graph':
            plt.figure(figsize=(20,10))
            plt.bar(x = dailyDF['Date'], height = dailyDF['Value'])
            plt.title('Daily sales summary')
            plt.xlabel('Date')
            plt.ylabel('Value(£)')
            plt.savefig('graph.jpg', bbox_inches = 'tight')
        else:
            dailyDF['Value'] = '£' + dailyDF['Value'].astype(str)
            html = dailyDF.to_html(index=False)
            print(html)
    else:
        weeklyDF = moneyDF
        weeklyDF['Week'] = weeklyDF['Date'].dt.isocalendar().week
        weeklyDF = weeklyDF.groupby('Week')['Value'].sum().reset_index()
        if  reportFormat == 'graph':
            plt.figure(figsize=(20,10))
            plt.bar(x = weeklyDF['Week'], height = weeklyDF['Value'], color = 'green')
            plt.title('Weekly sales summary')
            plt.xlabel('Week number')
            plt.ylabel('Value(£)')
            plt.savefig('graph.jpg', bbox_inches = 'tight')
        else:
            weeklyDF['Value'] = '£' + weeklyDF['Value'].astype(str)
            html = weeklyDF.to_html(index=False)
            print(html)


conn.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Week</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>£344855.67</td>
    </tr>
    <tr>
      <td>2</td>
      <td>£409428.28</td>
    </tr>
    <tr>
      <td>3</td>
      <td>£394936.90</td>
    </tr>
    <tr>
      <td>4</td>
      <td>£426632.16</td>
    </tr>
    <tr>
      <td>5</td>
      <td>£309490.76</td>
    </tr>
  </tbody>
</table>
<class 'str'>
